This Notebook prepares a sample provided by USC-ISI for graphing.  The data is in the file data_new.csv, which contains records of the form (datatype (bitrate or packetcount), flow, timestamp, value, timeoffset).  Time offset is an integer-valued offset of the timestamp from the lowest timestamp and is somewhat easier to deal with.  This notebook converts this single table into two tables, bitrate and packetcount, with fields (timeoffset, flow1, flow2,...), and writes them to the dashboard.

Step 1: Import the software we'll need.  Install the Google Visualization library (just a library that turns a table into a data structure Google charts can deal with and JSONifies it), and a lightweight client to send data to the dashboard.  test_client should be in the same directory as the Notebook.

In [137]:
!pip install --upgrade gviz_api
import gviz_api
import csv
import datetime
import json
import test_client

Requirement already up-to-date: gviz_api in /opt/conda/lib/python3.7/site-packages (1.9.0)


Read the CSV file

In [138]:
f = open('data-new.csv', 'r')
data_reader = csv.reader(f)
rows = [row for row in data_reader]
f.close()

Get the names of all the flows.  These will be the columns in the tables we'll spit out.

In [139]:
flows = list(set([row[1] for row in rows[1:]]))

The schema for both tables is the same: time as a number in column 0, the values for the flows of that timestamp in the remaining columns

In [140]:
schema = [('time', 'number')] + [(flow, 'number') for flow in flows]

We will have one table for each datatype in column 0 of the original table

In [141]:
tableNames = list(set([row[0] for row in rows[1:]]))

Actually compute the tables to be plotted.  First, find the unique times and sort  them in ascending order.  Then get the names of the columns as a list.  Initialize each table to 0 for all flows at each tiume.  Then go through the rows of the input table, and for each row, the output tableName is in column 0, the time is in column 4, and use that to derive the row number in the output table, the flow name is in column 1, and use that to find the column number in the output table, and the value is in column 3.  Collect all of these, and then put the values into the tables. 

In [142]:
tables = {}
times = list(set([int(row[4]) for row in rows[1:]]))
times.sort()
columnNames = [spec[0] for spec in schema]
for name in tableNames:
    tables[name] = [[time] + [0 for i in schema[1:]] for time in times]

values = [{"tableName": row[0], "rowNum": times.index(int(row[4])), "colNum": columnNames.index(row[1]), "value": int(row[3])} for row in rows[1:]]
for value in values:
    tables[value["tableName"]][value["rowNum"]][value["colNum"]] = value["value"]

Create a client and connect to the server.  The default url and path  is used.

In [143]:
client = test_client.Client()
client.connect()

Assign a room name to listen on.

In [144]:
roomName = 'isiRoom'

A utility to send a Table as JSON to the room, using the client.  This should migrate into the client.

In [145]:
def send_data_to_dashboard(table_name, data_table_as_JSON, client, room):
    table_record = '{"name": "%s", "table": %s}' % (table_name, data_table_as_JSON)
    client.send(table_record, room)

Create the Google Tables for each table name, using the schema, then load the data into the table, convert the table into JSON and use the utility just defined to send it to the table.

In [146]:
for name in tableNames:
    google_table = gviz_api.DataTable(schema)
    google_table.LoadData(tables[name])
    send_data_to_dashboard(name, google_table.ToJSon(), client, roomName)
    